# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [49]:
import nltk
nltk.download(['punkt', 'wordnet'])

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [50]:
# import libraries
import re
import pandas as pd 
from sqlalchemy import create_engine
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

from sklearn.metrics import confusion_matrix,classification_report
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier

In [51]:
# load data from database
engine = create_engine('sqlite:///DistasterResponse.db')
df = pd.read_sql('SELECT * FROM categorized_messages', engine)
df.head()

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [52]:
category_cols = [col for col in df.columns.tolist() if col not in ['id', 'message','original', 'genre']]
X = df.message.values
Y = df[category_cols]

### 2. Write a tokenization function to process your text data

In [53]:
def tokenize(text):
    text = re.sub('[\W_]+',' ', text)
    
    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()

    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)

    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [54]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier()))
])

def display_categories_report(y_pred, y_test):
    """
    Displays the classification report of each category predicted.
    """
    for idx, col in enumerate(y_test):
        print("Category: " + col)
        print(classification_report(y_test[col], y_pred[:, idx]))



### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [55]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.3, random_state = 1)

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [56]:
pipeline.fit(X_train, y_train)
y_pred = pipeline.predict(X_test)

In [57]:
display_categories_report(y_pred, y_test)

Category: related
             precision    recall  f1-score   support

          0       0.62      0.35      0.45      1868
          1       0.81      0.93      0.87      5937
          2       0.50      0.03      0.06        60

avg / total       0.76      0.79      0.76      7865

Category: request
             precision    recall  f1-score   support

          0       0.88      0.99      0.93      6511
          1       0.84      0.38      0.52      1354

avg / total       0.88      0.88      0.86      7865

Category: offer
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      7832
          1       0.00      0.00      0.00        33

avg / total       0.99      1.00      0.99      7865

Category: aid_related
             precision    recall  f1-score   support

          0       0.71      0.87      0.78      4577
          1       0.73      0.51      0.60      3288

avg / total       0.72      0.72      0.71      7865

Category: med

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 6. Improve your model
Use grid search to find better parameters. 

In [73]:
parameters = {
    'tfidf__use_idf': (True, False),
    'clf__estimator__n_estimators': [10, 20],
    'clf__estimator__min_samples_split': [2, 3, 4]
}
cv =  GridSearchCV(pipeline, param_grid=parameters)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [74]:
cv.fit(X_train, y_train)
y_pred = cv.predict(X_test)

In [75]:
display_categories_report(y_pred, y_test)

Category: related
             precision    recall  f1-score   support

          0       0.69      0.33      0.44      1868
          1       0.81      0.95      0.88      5937
          2       0.75      0.05      0.09        60

avg / total       0.78      0.80      0.77      7865

Category: request
             precision    recall  f1-score   support

          0       0.89      0.98      0.93      6511
          1       0.85      0.41      0.55      1354

avg / total       0.88      0.89      0.87      7865

Category: offer
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      7832
          1       0.00      0.00      0.00        33

avg / total       0.99      1.00      0.99      7865

Category: aid_related
             precision    recall  f1-score   support

          0       0.74      0.88      0.80      4577
          1       0.77      0.57      0.66      3288

avg / total       0.75      0.75      0.74      7865

Category: med

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

In [ ]:
import pickle
pickle.dump(cv, open('mymodel.pkl', 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.